In [90]:
import numpy as np
import csv
import math
import antropy as ant
import pandas as pd
from matplotlib import pyplot

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, confusion_matrix, roc_auc_score, 
                            precision_score, recall_score, f1_score, classification_report, 
                            roc_curve, plot_roc_curve, auc, precision_recall_curve, 
                            plot_precision_recall_curve, average_precision_score)
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import StackingClassifier, VotingClassifier, BaggingClassifier

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold


### Data Processing

In [91]:
begin, end = 1, 61 # (begin is inclusive, end is exclusive)
num_people = 14
count_samples = {
    "active": 8,
    "meditate": 8,
    "neutral": 8
}

class Sample:
    def __init__(self):
        self.data = {
            'RawEEG': [],
            'Alpha': [],
            'Low Beta': [],
            'High Beta': [],
            'Gamma': [],
            'Theta': [],
            'Delta': [],
            'Meditation': [],
            'Attention': []
        }

    def recordDataPoint(self, RawEEG, Attention, Meditation, Alpha, Delta, Theta, LowBeta, HighBeta, Gamma):
        self.data['RawEEG'].append(float(RawEEG))
        self.data['Attention'].append(float(Attention))
        self.data['Meditation'].append(float(Meditation))
        self.data['Alpha'].append(float(Alpha))
        self.data['Delta'].append(float(Delta))
        self.data['Theta'].append(float(Theta))
        self.data['Low Beta'].append(float(LowBeta))
        self.data['High Beta'].append(float(HighBeta))
        self.data['Gamma'].append(float(Gamma))

    '''
    Record a line of data from the CSV output, which takes form RawEEG, Alpha, Delta, Gamma, Low Beta, High Beta, Theta, Attention, Meditation

    '''
    def recordDataLine(self, line):
        self.recordDataPoint(line[0], line[7], line[8], line[1], line[2], line[6], line[4], line[5], line[3])
    
    def getEEG(self):
        return self.data['RawEEG']
    
    def getAttention(self):
        return self.data["Attention"]
    
    def getMeditation(self):
        return self.data["Meditation"]
    
    def getAlpha(self):
        return self.data["Alpha"]
    
    def getDelta(self):
        return self.data["Delta"]
    
    def getTheta(self):
        return self.data["Theta"]
    
    def getLowBeta(self):
        return self.data["Low Beta"]
    
    def getHighBeta(self):
        return self.data["High Beta"]
    
    def getGamma(self):
        return self.data["Gamma"]

    def get(self, key):
        return self.data[key]

    '''
    Filter out all outliers, as defined by being outside 3*std from the mean, and replace with mean of the samples around them
    '''
    def filter_outliers(self):
        sampleBad = False
        for key in ['RawEEG', 'Alpha', 'Theta', 'Low Beta', 'High Beta', "Gamma", 'Delta']:
            data = self.data[key]
            
            filtered = []

            iqr = np.subtract(*np.percentile(data, [75, 25]))
            med = np.median(data)

            for x in data:
                
                if (med - 1.5*iqr > x) or (med + 1.5*iqr < x) or abs(x - np.mean(data)) > 2 * np.std(data):
                    filtered.append(med)
                    # filtered.append(np.median(data[max(0, i-5):i] + data[i+1:min(len(data), i+5)]))
                else:
                    filtered.append(x)
                    
            self.data[key] = filtered
        return sampleBad

In [92]:
# {personNum : {state: [sampleNums]}}
# 0 = key for throwing away all samples of that state

badSamples = {
    1: {"active": [5], "neutral": [2], "meditate": []},
    2: {"active": [0], "neutral": [0], "meditate": [0]},
    3: {"active": [1, 4], "neutral": [1], "meditate": [5, 6, 7, 8]},
    4: {"active": [2], "neutral": [1, 7], "meditate": [1, 8]},
    5: {"active": [], "neutral": [], "meditate": []}, 
    6: {"active": [], "neutral": [2, 6], "meditate": []},
    7: {"active": [5], "neutral": [4, 6, 7], "meditate": [1, 3, 4, 8]}, 
    8: {"active": [5], "neutral": [1], "meditate": [5, 8]}, 
    9: {"active": [], "neutral": [], "meditate": []}, 
    10: {"active": [6, 8], "neutral": [4, 5, 6], "meditate": []},
    11: {"active": [4], "neutral": [4, 8], "meditate": [1, 2, 3, 5, 7]},
    12: {"active": [2, 3, 8], "neutral": [0], "meditate": [6]}, 
    13: {"active": [], "neutral": [8], "meditate": []},
    14: {"active": [4, 5, 8], "neutral": [0], "meditate": [1, 2, 8]}
}

In [93]:
data = []
dataLabels = []

def transcribeFileToSample(personN: int, sampleN: int, state: str, X, y, outlierFiltering = True):
    sample_data = Sample()

    with open("data/all_data/" + state + "_" + str(personN) + "_" + str(sampleN) + ".csv") as f:
        reader = csv.reader(f)

        header = next(reader)
        
        for row in reader:
            sample_data.recordDataLine(row)

        if (outlierFiltering):   
            if (0 not in badSamples[personN][state] and sampleN not in badSamples[personN][state]):

                for key in sample_data.data:
                    sample_data.data[key] = sample_data.data[key][begin:end]

                sample_data.filter_outliers()
                X.append(sample_data)
                y.append(state)
        else:
            X.append(sample_data)
            y.append(state)

for person in range(num_people):
    for state in count_samples:
        for i in range(count_samples[state]):
            transcribeFileToSample(person + 1, i + 1, state, data, dataLabels)

In [94]:
dataExtracted = []

def safety_check(x):
    if math.isnan(x): return 0
    if math.isinf(x): return 99999999999
    return x

for point in data:
    extractedPoint = []

    extractedPoint.append(np.mean(point.getAlpha()))
    extractedPoint.append(np.mean(point.getLowBeta()))
    extractedPoint.append(np.mean(point.getHighBeta())) 
    extractedPoint.append(np.mean(point.getGamma())) 
    extractedPoint.append(np.mean(point.getTheta()))
    extractedPoint.append(np.std(point.getHighBeta())) 
    extractedPoint.append(np.std(point.getGamma()))
    extractedPoint.append(np.std(point.getDelta()))
    extractedPoint.append(safety_check(ant.sample_entropy(point.getDelta())))
    extractedPoint.append(np.std(point.getLowBeta())) 
    extractedPoint.append(np.std(point.getTheta()))
    
    # extractedPoint.append(safety_check(ant.spectral_entropy(point.getEEG(), sf=1)))
    # extractedPoint.append(np.mean(point.getDelta()))
    # extractedPoint.append(np.std(point.getAlpha())) 

    dataExtracted.append(extractedPoint)

X_train, X_test, y_train, y_test = train_test_split(dataExtracted, dataLabels, test_size=0.2, random_state=13, stratify=dataLabels)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 13)

### Hyperparameter Training for Individual Classifiers

In [95]:
models = [
    Pipeline([('ss', StandardScaler()), ('classifier', LogisticRegression(max_iter=1000))]), 
    Pipeline([('ss', StandardScaler()), ('classifier', KNeighborsClassifier())]),
    Pipeline([('classifier', GaussianNB())]),
    Pipeline([('classifier', BernoulliNB())]),
    Pipeline([('ss', StandardScaler()), ('classifier', DecisionTreeClassifier())]),
    Pipeline([('ss', StandardScaler()), ('classifier', RandomForestClassifier())]),
    # Pipeline([('ss', StandardScaler()), ('classifier', XGBClassifier())]),
    Pipeline([('ss', StandardScaler()), ('classifier', GradientBoostingClassifier())]),
    Pipeline([('ss', StandardScaler()), ('classifier', SVC(probability=True, max_iter=1000))]),
    Pipeline([('ss', StandardScaler()), ('classifier', AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(criterion="entropy", max_depth=None, 
                                                max_features=None, min_samples_leaf=1, 
                                                min_samples_split=2, random_state=0)
    ))]),
]

model_grids = [
               [{'classifier__C':[1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 5, 1e1, 5e1, 1e2, 5e2, 1e3],
                 'classifier__random_state':[0]}],                                 #logistic regression
               
               [{'classifier__n_neighbors':[5,7,9,11, 13, 15, 17, 19], 
                 'classifier__metric': ['euclidean', 'manhattan', 'minkowski']}],  #KNN
               
               [{'classifier__var_smoothing': [1e-10, 1e-09, 1e-8, 1e-7]}],        #GaussianNB

               [{'classifier__alpha': [1e-2, 1e-1, 1, 1e1, 1e2]}],                 #BernoulliNB

               [{'classifier__criterion':['gini','entropy'],
                 'classifier__random_state':[0], 
                 'classifier__max_depth' : [3, 5, 8, 10, 15, None], 
                 'classifier__min_samples_split' : [1.0,2,5,10,15,30],
                 'classifier__min_samples_leaf': [1,2,5,10], 
                 'classifier__max_features': ['log2', 'sqrt', None]}],
#Decision Tree
               
               [{'classifier__criterion':['gini','entropy'],
                 'classifier__n_estimators': [1000],
                 'classifier__random_state':[0], 
                 'classifier__max_depth' : [3, 5, 8, 10, 15, None], 
                 'classifier__min_samples_split' : [1.0,2,5,10,15,30],
                 'classifier__min_samples_leaf': [1,2,5,10], 
                 'classifier__max_features': ['log2', 'sqrt', None]}],             #Random Forest

              #  [{'classifier__n_estimators':[1000],
              #    'classifier__criterion':['gini','entropy'],
              #    'classifier__random_state':[0],
              #    'classifier__max_depth': [3, 5, 8, 10, 15, 30],
              #    'classifier__min_child_weight': [2,4,6,8,10],
              #    'classifier__gamma': [0, 0.1, 0.2, 0.3],
              #    'classifier__reg_alpha':[0, 0.001, 0.005, 0.01, 0.05],
              #    'classifier__colsample_bytree': [0.6, 0.7, 0.8, 0.9],
              #    'classifier__eta': [0.1, 0.2, 0.3, 0.4, 0.5]}],                   #XGBoost

                [{'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2], 
                 'classifier__n_estimators': [1000],
                 'classifier__random_state':[0],
                 'classifier__max_depth' : [5, 8, 15, None], 
                 'classifier__min_samples_split' : [1.0,2,5,10],
                 'classifier__min_samples_leaf': [1,2,5,10], 
                 'classifier__max_features': ['log2', 'sqrt', 'auto', 'None']}],   #Gradient Bossting Decision Tree
                        

               [{'classifier__C':[1e-1, 1, 1e1] ,
                 'classifier__random_state':[0],
                 'classifier__kernel': ['rbf', 'poly']
                }],                                                                #SVM
               
               [{'classifier__n_estimators' : [800, 1000, 1200], 
                 'classifier__learning_rate' : [1e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 1e1],
                 'classifier__random_state':[0]}]                                  #AdaBoost
]

In [96]:
# Commented out so it doesn't run every time I run all and waste a lot of time

# for i,j in zip(models, model_grids):
#     grid = RandomizedSearchCV(estimator=i, param_distributions=j, n_iter = 100, scoring='f1_weighted', cv = skf, n_jobs=-1)
#     grid.fit(X_train, y_train)
#     best_f1 = grid.best_score_
#     best_param = grid.best_params_
#     print('{}:\nBest F1 : {:.4f}'.format(i.steps[-1],best_f1))
#     print('Best Parameters : ',best_param)
#     print('')
#     print('----------------')
#     print('')

# Logistic Regression: F1 0.567, C=500.0
# KNN: F1 0.686, n_neighbors=5, metric="manhattan"
# GaussianNB: F1 0.540, var_smoothing = 1e-10
# BernoulliNB: F1 0.194, alpha=0.01
# Decision Tree: F1 0.694, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=None, criterion=entropy
# Random Forest: F1 idk, n_estimators: 1000, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=entropy
# Gradient Boosting: F1 0.767, n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=8, learning_rate=0.2
# SVC: F1 0.67, kernel=rbf, C=10.0
# Ada Boost: F1: 0.603, n_estimators=800, learning_rate=0.001

In [97]:
# list of all models with their tuned hyperparameters
models_tuned = [
    Pipeline([('ss', StandardScaler()), ('classifier', LogisticRegression(max_iter=1000, C=500.0))]), 
    Pipeline([('ss', StandardScaler()), ('classifier', KNeighborsClassifier(n_neighbors=5, metric="manhattan"))]),
    Pipeline([('classifier', GaussianNB(var_smoothing=1e-10))]),
    Pipeline([('classifier', BernoulliNB(alpha=0.01))]),
    Pipeline([('ss', StandardScaler()), ('classifier', DecisionTreeClassifier(min_samples_split=2, min_samples_leaf=2, 
                                                                            max_features="log2", criterion="entropy"))]),
    Pipeline([('ss', StandardScaler()), ('classifier', RandomForestClassifier(n_estimators=1000, min_samples_split=2, 
                                                                            min_samples_leaf=1, max_features="sqrt", criterion="entropy"))]),
    Pipeline([('ss', StandardScaler()), ('classifier', GradientBoostingClassifier(n_estimators=1000, min_samples_split=2,
                                                                                min_samples_leaf=2, max_features="log2",
                                                                                max_depth=8, learning_rate=0.2))]),
    Pipeline([('ss', StandardScaler()), ('classifier', SVC(probability=True, max_iter=1000, kernel="rbf", C=10.0))]),
    Pipeline([('ss', StandardScaler()), ('classifier', AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(criterion="entropy", max_depth=None, 
                                                max_features=None, min_samples_leaf=1, 
                                                min_samples_split=2, random_state=0), n_estimators=800, learning_rate=0.001))])
]

### Statistics/Scores for Individual Classifiers

In [98]:
tuned_model_stats = []

for m in range(len(models_tuned)):
    model_stats = []
    model = models_tuned[m]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_test = np.array(y_test)

    accuracies = cross_val_score(estimator = model, X = X_train, y = y_train, cv = skf)   #K-Fold Validation
    test_acc = accuracy_score(y_test, y_pred)
    cr = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average= 'weighted')
    recall = recall_score(y_test, y_pred, average= 'weighted')
    f1 = f1_score(y_test, y_pred, average= 'weighted')

    predicted_probab = model.predict_proba(X_test)
    predicted_probab = predicted_probab[:, 1]
   
    print(type(models_tuned[m][-1]).__name__ , ':')
    
    print('Accuracy Score: {:.4f}'.format(test_acc))
    print("K-Fold Validation Mean Accuracy: {:.4f} %".format(accuracies.mean()*100))
    print("Classification report: ")
    print(cr)
    
    print('-----------------------------------')
    print('')
    model_stats.append(type(models_tuned[m][-1]).__name__)
    model_stats.append(accuracies.mean())
    model_stats.append(test_acc)
    model_stats.append(precision)
    model_stats.append(recall)
    model_stats.append(f1)
    tuned_model_stats.append(model_stats)

LogisticRegression :
Accuracy Score: 0.5918
K-Fold Validation Mean Accuracy: 58.4615 %
Classification report: 
              precision    recall  f1-score   support

      active       0.61      0.61      0.61        18
    meditate       0.75      0.71      0.73        17
     neutral       0.40      0.43      0.41        14

    accuracy                           0.59        49
   macro avg       0.59      0.58      0.58        49
weighted avg       0.60      0.59      0.60        49

-----------------------------------

KNeighborsClassifier :
Accuracy Score: 0.6735
K-Fold Validation Mean Accuracy: 69.2308 %
Classification report: 
              precision    recall  f1-score   support

      active       0.67      0.67      0.67        18
    meditate       0.80      0.71      0.75        17
     neutral       0.56      0.64      0.60        14

    accuracy                           0.67        49
   macro avg       0.68      0.67      0.67        49
weighted avg       0.68      0.6

In [99]:
df_tuned = pd.DataFrame(tuned_model_stats, columns= ['Model','Cross-val Acc','Test Accuracy','Precision','Recall', 'F1'])
df_tuned.sort_values(by= ['F1'], inplace= True, ascending= False)
df_tuned

,Model,Cross-val Acc,Test Accuracy,Precision,Recall,F1
5,RandomForestClassifier,0.717949,0.857143,0.868580,0.857143,0.859472
6,GradientBoostingClassifier,0.753846,0.816327,0.834334,0.816327,0.821898
8,AdaBoostClassifier,0.610256,0.775510,0.789571,0.775510,0.772656
1,KNeighborsClassifier,0.692308,0.673469,0.683163,0.673469,0.676531
4,DecisionTreeClassifier,0.661538,0.673469,0.721978,0.673469,0.672233
7,SVC,0.682051,0.632653,0.650170,0.632653,0.639711
2,GaussianNB,0.569231,0.612245,0.658050,0.612245,0.600186
0,LogisticRegression,0.584615,0.591837,0.598980,0.591837,0.595036
3,BernoulliNB,0.364103,0.367347,0.134944,0.367347,0.197380


add SGDClassifier and/or MLPClassifier??

### Ensemble

In [100]:
indiv_classifiers = {
    # ('log reg', Pipeline([('ss', StandardScaler()), ('classifier', LogisticRegression(max_iter=1000, C=500.0))])), 
    # ('gaussianNB', Pipeline([('classifier', GaussianNB(var_smoothing=1e-10))])),
    # ('bernoulliNB', Pipeline([('classifier', BernoulliNB(alpha=0.01))])),
    ('decistion tree', Pipeline([('ss', StandardScaler()), ('classifier', DecisionTreeClassifier(min_samples_split=2, min_samples_leaf=2, 
                                                                            max_features="log2", criterion="entropy"))])), 
    ('random forest', Pipeline([('ss', StandardScaler()), ('classifier', RandomForestClassifier(n_estimators=1000, min_samples_split=2, 
                                                                            min_samples_leaf=1, max_features="sqrt", criterion="entropy"))])), 
    ('gradient boosting', Pipeline([('ss', StandardScaler()), ('classifier', GradientBoostingClassifier(n_estimators=1000, min_samples_split=2,
                                                                                min_samples_leaf=2, max_features="log2",
                                                                                max_depth=8, learning_rate=0.2))])), 
    ('svc', Pipeline([('ss', StandardScaler()), ('classifier', SVC(probability=True, max_iter=1000, kernel="rbf", C=10.0))])), 
    ('ada boost', Pipeline([('ss', StandardScaler()), ('classifier', AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(criterion="entropy", max_depth=None, 
                                                max_features=None, min_samples_leaf=1, 
                                                min_samples_split=2, random_state=0), n_estimators=800, learning_rate=0.001))]))
}

final_est = final_estimator=RandomForestClassifier(criterion='entropy', n_estimators= 1000, random_state= 0)


In [101]:
stackingClf = StackingClassifier(estimators=indiv_classifiers, final_estimator=final_est)

stackingClf.fit(X_train, y_train)
y_pred = stackingClf.predict(X_test)
y_test = np.array(y_test)

accuracies = cross_val_score(estimator = stackingClf, X = X_train, y = y_train, cv = skf)   #K-Fold Validation
test_acc = accuracy_score(y_test, y_pred)
cr = classification_report(y_test, y_pred)
precision = precision_score(y_test, y_pred, average= 'weighted')
recall = recall_score(y_test, y_pred, average= 'weighted')
f1 = f1_score(y_test, y_pred, average= 'weighted')

model_stats = []
model_stats.append(type(stackingClf).__name__)
model_stats.append(accuracies.mean())
model_stats.append(test_acc)
model_stats.append(precision)
model_stats.append(recall)
model_stats.append(f1)
tuned_model_stats.append(model_stats)

print('Accuracy Score: {:.4f}'.format(test_acc))
print("K-Fold Validation Mean Accuracy: {:.4f} %".format(accuracies.mean()*100))
print("Classification report: ")
print(cr)

Accuracy Score: 0.7755
K-Fold Validation Mean Accuracy: 71.7949 %
Classification report: 
              precision    recall  f1-score   support

      active       0.70      0.78      0.74        18
    meditate       1.00      0.82      0.90        17
     neutral       0.67      0.71      0.69        14

    accuracy                           0.78        49
   macro avg       0.79      0.77      0.78        49
weighted avg       0.79      0.78      0.78        49



In [102]:
votingClf = VotingClassifier(estimators=indiv_classifiers, voting='soft', weights = [1, 1.5, 1.4, 1, 1])
# tree, forest, gradient, svc, ada

votingClf.fit(X_train, y_train)
y_pred = votingClf.predict(X_test)
y_test = np.array(y_test)

accuracies = cross_val_score(estimator = votingClf, X = X_train, y = y_train, cv = skf, n_jobs=-1)   #K-Fold Validation
test_acc = accuracy_score(y_test, y_pred)
cr = classification_report(y_test, y_pred)
precision = precision_score(y_test, y_pred, average= 'weighted')
recall = recall_score(y_test, y_pred, average= 'weighted')
f1 = f1_score(y_test, y_pred, average= 'weighted')

model_stats = []
model_stats.append(type(votingClf).__name__)
model_stats.append(accuracies.mean())
model_stats.append(test_acc)
model_stats.append(precision)
model_stats.append(recall)
model_stats.append(f1)
tuned_model_stats.append(model_stats)

print('Accuracy Score: {:.4f}'.format(test_acc))
print("K-Fold Validation Mean Accuracy: {:.4f} %".format(accuracies.mean()*100))
print("Classification report: ")
print(cr)

Accuracy Score: 0.8367
K-Fold Validation Mean Accuracy: 68.7179 %
Classification report: 
              precision    recall  f1-score   support

      active       0.80      0.89      0.84        18
    meditate       1.00      0.82      0.90        17
     neutral       0.73      0.79      0.76        14

    accuracy                           0.84        49
   macro avg       0.84      0.83      0.83        49
weighted avg       0.85      0.84      0.84        49



In [103]:
df_tuned = pd.DataFrame(tuned_model_stats, columns= ['Model','Cross-val Acc','Test Accuracy','Precision','Recall', 'F1'])
df_tuned.sort_values(by= ['F1'], inplace= True, ascending= False)
df_tuned

,Model,Cross-val Acc,Test Accuracy,Precision,Recall,F1
5,RandomForestClassifier,0.717949,0.857143,0.868580,0.857143,0.859472
10,VotingClassifier,0.687179,0.836735,0.850340,0.836735,0.839458
6,GradientBoostingClassifier,0.753846,0.816327,0.834334,0.816327,0.821898
9,StackingClassifier,0.717949,0.775510,0.794558,0.775510,0.781085
8,AdaBoostClassifier,0.610256,0.775510,0.789571,0.775510,0.772656
1,KNeighborsClassifier,0.692308,0.673469,0.683163,0.673469,0.676531
4,DecisionTreeClassifier,0.661538,0.673469,0.721978,0.673469,0.672233
7,SVC,0.682051,0.632653,0.650170,0.632653,0.639711
2,GaussianNB,0.569231,0.612245,0.658050,0.612245,0.600186
0,LogisticRegression,0.584615,0.591837,0.598980,0.591837,0.595036
